In [1]:
import numpy as np

In [2]:
def fit(x_train,y_train):
    result = {}
    class_value = set(y_train)
    for current_class in class_value:
        result[current_class] = {}
        result["total_data"] = len(y_train)
        current_class_row = (y_train==current_class)
        x_train_current = x_train[current_class_row]
        y_train_current = y_train[current_class_row]
        num_feature = x_train.shape[1]
        result[current_class]["total_count"] = len(y_train_current)
        for j in range(1,num_feature+1):
            result[current_class][j] = {}
            all_possible_values = set(x_train[:,j-1])
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (x_train_current[:,j-1]==current_value).sum()
    return result

In [3]:
num_feature = 5
for j in range(1,num_feature+1):
    print(j)

1
2
3
4
5


In [4]:
def probabilty(dictionary,x,current_class):
    output = np.log(dictionary[current_class]['total_count']) - np.log(dictionary['total_data'])
    num_feature = len(dictionary[current_class].keys()) - 1
    for j in range(1,num_feature+1):
        xj = x[j-1]
        count_current_class_with_value_xj = dictionary[current_class][j][xj] + 1
        count_current_class = dictionary[current_class]['total_count'] + len(dictionary[current_class][j].keys())
        current_xj_probabilty =  np.log(count_current_class_with_value_xj)-np.log(count_current_class)
        output = output+current_xj_probabilty
    return output

In [5]:
def predictsinglepoint(dictionary,x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if(current_class=="total_data"):
            continue
        p_current_class = probabilty(dictionary, x,current_class)
        if(first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
    return best_class
            

In [6]:
def predict(dictionary,x_test):
    y_pred = []
    for x in x_test:
        x_class = predictsinglepoint(dictionary, x)
        y_pred.append(x_class)
    return y_pred

In [7]:
def markedlable(column):
    second_limit = column.mean()
    first_limit = second_limit*0.5
    third_limit = second_limit*1.5
    for i in range(0,len(column)):
        if(column[i] < first_limit):
            column[i] = 0
        elif(column[i] < second_limit):
            column[i] = 1
        elif(column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column

In [8]:
from sklearn import datasets
iris = datasets.load_iris()
x = iris.data
y = iris.target

In [9]:
for i in range(0,x.shape[-1]):
    x[:,i] = markedlable(x[:,i])

In [10]:
from sklearn import model_selection
x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size = 0.25,random_state = 0)

In [11]:
dictionary = fit(x_train,y_train)

In [12]:
y_pred = predict(dictionary,x_test)

In [13]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

    accuracy                           0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]


In [14]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.76      1.00      0.86        16
           2       1.00      0.67      0.80         9

    accuracy                           0.87        38
   macro avg       0.92      0.84      0.86        38
weighted avg       0.90      0.87      0.87        38

[[11  2  0]
 [ 0 16  0]
 [ 0  3  6]]


In [15]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        13
           1       0.00      0.00      0.00        16
           2       0.36      1.00      0.53         9

    accuracy                           0.53        38
   macro avg       0.45      0.62      0.48        38
weighted avg       0.43      0.53      0.44        38

[[11  2  0]
 [ 0  0 16]
 [ 0  0  9]]
